# COMP0164: Digital Finance
## Group Coursework

If you have set up an **`Anaconda`** environment, don't forget to activate it using:

> conda activate digital_finance_env

Below are the imports that you will need for the coursework:

In [2]:
import numpy as np
import pandas as pd
import numpy_financial as npf
from scipy import stats
import datetime

import yfinance as yf
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.efficient_frontier import EfficientFrontier

import matplotlib.pyplot as plt
%matplotlib inline

<div class="alert alert-danger">
<b>WARNING: </b>

For question 5, you may want to use **pypfopt** package to construct the efficient frontier. If you use Python>=3.6, the plotting functionality of the **pypfopt** package may not work. Please use the function defined below to plot EfficientFrontier object.
</div>

In [2]:
import copy

def plot_efficient_frontier(ef: EfficientFrontier, points: int=100) -> None:
    fig, ax = plt.subplots()

    # Generate range of returns
    ef_minvol = copy.deepcopy(ef)
    ef_maxret = copy.deepcopy(ef)
    ef_minvol.min_volatility()
    min_ret = ef_minvol.portfolio_performance()[0]
    max_ret = ef_maxret._max_return()
    ef_param_range = np.linspace(min_ret, max_ret - 0.0001, points)

    # Plot efficient frontier from EfficientFrontier object
    mus, sigmas = [], []
    for param_value in ef_param_range:
        ef_i = copy.deepcopy(ef)
        try:
            ef_i.efficient_return(param_value)
        except:
            raise ValueError("pypfopt experiences problems with portfolio optimization.")
        ret, sigma, _ = ef_i.portfolio_performance()
        mus.append(ret)
        sigmas.append(sigma)
    
    # Generate axis and plot
    ax.plot(sigmas, mus, label="Efficient Frontier")
    ax.legend()
    ax.set_xlabel("Volatility")
    ax.set_ylabel("Return")
    plt.show()

## To-Do List

- [ ] Read Coursework Instructions
- [ ] Read Scenario Setting
- [ ] Question 1
  - [ ] Part a.)
  - [ ] Part b.)
  - [ ] Part c.)
  - [ ] Part d.)
  - [ ] Part e.)
  - [ ] Part f.)
  - [ ] Part g.)
- [ ] Question 2
  - [ ] Part a.)
  - [ ] Part b.)
  - [ ] Part c.)
  - [ ] Part d.)
  - [ ] Part e.)
  - [ ] Part f.)
  - [ ] Part g.)
- [ ] Question 3
  - [ ] Part a.)
  - [ ] Part b.)
  - [ ] Part c.)
  - [ ] Part d.)
  - [ ] Part e.)
  - [ ] Part f.)
  - [ ] Part g.)
  - [ ] Part h.)
  - [ ] Part i.)
- [ ] Question 4
  - [ ] Part a.)
  - [ ] Part b.)
  - [ ] Part c.)
  - [ ] Part d.)
  - [ ] Part e.)
  - [ ] Part f.)
- [ ] Question 5
  - [ ] Part a.)
  - [ ] Part b.)
  - [ ] Part c.)
  - [ ] Part d.)
- [ ] Question 6
  - [ ] Part a.)
  - [ ] Part b.)

## Scenario Setting
Conglomerates are companies that hold assets including other companies and involve themselves directly into the operations of their subsidiaries. Therefore, it is common for conglomerates to evaluate the business projects of their holdings to understand the returns of these projects.

Suppose that you are an executive director working for a conglomerate called Accretion and your job is to assist various operations of the subsidiaries that exist under Accretion, and then report back to the stakeholders.

## Question 1 [16 marks]

You areasked to evaluate businesses and investment ideas on behalf of Accretion as part of their portfolio reshuffling.

Boogle is one of the conglomerate's largest holdings. During the latest earnings report, Boogle's CFO announced an investment of \$150 million for a new business expansion project. The project is planned to be financed with an \$100 million public offering of a 10-year debt and the remainder with an equity offering. You have collected the information necessary to evaluate this project in Exhibits 1 and 2.

### Exhibit 1: Relevant Information for Analysis
| | |
| --- | --- |
| Equity risk premium | 4.93% |
| Risk-free rate of interest | 4.3% |
| Market value of Boogle's debt | \$1.0 billion |
| Market value of Boogle's equity | \$2.6 billion |
| Boogle's equity beta | 1.3 |
| Boogle's before-tax cost of debt | 9.4% |
| Corporate tax rate | 37.5% |


### Exhibit 2: Estimated Project Financials
|     | Year 1 | Year 2 | Year 3 |
| --- | --- | --- | --- |
| Revenue | 96.7 | 111.2 | 115.44 |
| Operating Costs | 32  | 36  | 38  |
| Depreciation | 16  | 16  | 16  |

1 a.) Calculate the weighted average cost of capital of Boogle prior to its new project investment. **\[3 marks\]**

Weighted average cost of capital is calculated according to this formula:

$$WACC = \frac{E}{V}\ * R_{e} + \frac{D}{V}\ * R_{d} * (1 - T_{c})$$

Where:

$E$ = Market value Equity

$D$ = Market value of Debt

$V=E+D$ = Total market value

$R_{e}$ = Cost of Equity

$R_{d}$ = Cost of Debt

$T_{c}$ = Corporate tax rate

In [12]:
# Solution
def cost_of_equity(rf:float, beta:float, equity_risk_premium:float) -> float:
    return (rf + beta*equity_risk_premium)

def WACC(E:float, D:float, cost_of_equity:float, cost_of_debt:float, tax_rate:float) -> float:
    return ((E/(E+D))*cost_of_equity + (D/(E+D))*cost_of_debt*(1-tax_rate))

D = 1.0
E = 2.6
equity_risk_premium = 0.0493
rf = 0.043
beta = 1.3
cost_of_debt = 0.094
tax_rate = 0.375

cost_of_equity = cost_of_equity(rf, beta, equity_risk_premium)
WACC_pre_investment = np.round(WACC(E, D, cost_of_equity, cost_of_debt, tax_rate), decimals=5)


print("The WACC of Boogle before its new project investment:", WACC_pre_investment*100 ,"%")

The WACC of Boogle before its new project investment: 9.366 %


1 b.) Find Boogle's asset beta prior to the new project. **\[2 marks\]**

Company's asset (unlevered) beta can be calculated according to this formula from the lecture notes:


$$\beta_{A} = \beta_{D} * \frac{D}{V}\ +  \beta_{E} * \frac{E}{V}\$$

Where:

$D$ = debt proportion (by value)

$E$ = equity proportion (by value)

$V$ = $E$ + $D$

$\beta_{A}$ = unlevered beta of the firm

$\beta_{E}$ = levered beta of the firm

$\beta_{D}$ = debt beta of the firm


In [14]:
# Solution

#This still assumes b_d = 0, but incorporates the effect of tax. Not sure which one to use
def asset_beta_tax(equity_beta:float, tax_rate:float, D: float, E:float) -> float:
    return (equity_beta/(1+(1 - tax_rate)*(D/E)))

def asset_beta(equity_beta:float, D: float, E:float) -> float:
    return (E/(E+D)*equity_beta + D/(E+D)*0)

b_A = np.round(asset_beta_tax(beta, tax_rate, D, E), decimals=3)
print(b_A)
b_A = np.round(asset_beta(beta, D, E), decimals=3)
print("Asset beta before the new project:", b_A)

1.048
Asset beta before the new project: 0.939


1 c.) Assuming the new project has the same asset beta as the Boogle company in b.), find the the project equity beta. **\[2 marks\]**

To calculate the project's equity beta, we can use the formula from the lecture slides:

$$\beta_{E} = \beta_{A} + (\beta_{A} - \beta_{D}) * \frac{D}{E}\$$

Where:

$D$ = debt proportion (by value)

$E$ = equity proportion (by value)

$\beta_{E}$ = levered beta of the firm

$\beta_{A}$ = unlevered beta of the firm


In [20]:
# Solution
def equity_beta(D:float, E:float, asset_beta:float, tax_rate:float) -> float:
    return (asset_beta + (asset_beta - 0)*(D/E))

b_A_project = b_A
D_project = 0.1
E_project = 0.05

b_E_project = np.round(equity_beta(D_project, E_project, b_A_project, tax_rate), decimals=3)
print("The new project equity beta: ", b_E_project)

The new project equity beta:  2.817


1 d.) The formula for project after-tax free cash flow at time t is

$$ FCF = (\text{Revenue}-\text{Operating Costs}-\text{Depreciation})\times(1-\text{Tax Rate})+\text{Depreciation}.$$

Define a Python function to calculate the project FCFs and demonstrate that the after-tax free cash flow generated for the next three years are $46.4 million, $53 million, and $54.4 million respectively. **\[2 marks\]**

In [26]:
# Solution
def FCF(revenue: float, op_costs: float, depreciation: float, tax_rate: float) -> float: 
    return (revenue - op_costs - depreciation) * (1 - tax_rate) + depreciation

revenue = [96.7, 111.2, 115.44]
op_costs = [32, 36, 38]
depreciation = [16, 16, 16]

for(k, r) in enumerate(revenue):
    print(f"Year {k + 1} FCF: ${FCF(r, op_costs[k], depreciation[k], tax_rate):.1f} million")

Year 1 FCF: $46.4 million
Year 2 FCF: $53.0 million
Year 3 FCF: $54.4 million


1 e.) Find the project NPV and IRR with the next three years after-tax free cash flow given in d.).**\[2 marks\]**

Since the project and the firm overall have a similar risk profile (equal asset betas), we can use pre-investment WACC for NPV calculation

In [35]:
# Solution
cashflows = [-150, 46.4, 53, 54.4]
print("NPV value with rate", WACC_pre_investment, "% :", round(npf.npv(WACC_pre_investment, cashflows)))

irr = round(npf.irr(cashflows),3)
print("IRR value with three terms:", irr, "%")

NPV value with rate 0.09366 % : -22
IRR value with three terms: 0.012 %


You also aim to add dividend-paying stock to the conglomerate's portfolio of holdings. You begin by reviewing the following candidates:

### Candidate 1:
Atat Steel is in the steel manufacturing sector with a required rate of return of 7.35%. You estimate that if the economy is booming, the company’s current annual dividend of $0.7 per share will grow 11.5% a year for the next four years and then stabilize at a 3.5% growth rate a year indefinitely. However, if the economy falls into a recession, then Atat Steel will not likely experience the elevated 11.5% short-run growth and instead will grow by 3.5% indefinitely.

### Candidate 2:
GT&T company is a mature company with a stable capital structure. The company had an EPS of $2 in 2023. The earnings in the next year without the additional planned investments are expected to remain at $2. The earnings retention ratio is 0.60. The company is expected to earn an ROE of 15% on its investments, and the required rate of return is 12%. Assume that all dividends are paid at the end of the year.

1 f.) Use the discount dividend method and find the current value of Atat Steel stock under both economic conditions. **\[3 marks\]**

In [41]:
# Solution
current_dividend = 0.7  
boom_growth_rate = 0.115  
recession_growth_rate = 0.035 
required_rate_of_return = 0.0735 

def DDM_two_periods(current_dividend: float, boom_growth_rate: float, stable_growth_rate: float, required_rate_of_return: float, boom_years: int) -> float:
    q = (1+boom_growth_rate)/(1+required_rate_of_return)
    boom_period = (q*current_dividend)*(1-q**boom_years)/(1-q)
    k = (1+ stable_growth_rate)/(1+required_rate_of_return)
    stable_period = (k*current_dividend*q**boom_years)/(1-k)

    return boom_period + stable_period

def DDM_indefinite(current_dividend: float, growth_rate: float, required_rate_of_return: float) -> float:
    return current_dividend * (1 + growth_rate) / (required_rate_of_return - growth_rate)

print("Current Stock Price (Recession): $", round(DDM_indefinite(current_dividend, recession_growth_rate, required_rate_of_return), 2))
print("Current Stock Price (Booming Economy): $", round(DDM_two_periods(current_dividend, boom_growth_rate, recession_growth_rate, required_rate_of_return, 4), 2))


Current Stock Price (Recession): $ 18.82
Current Stock Price (Booming Economy): $ 24.98


1 g.) Calculate GT&T's sustainable growth rate, find the value of the company's stock at the beginning of 2024 and determine the company's present value of growth opportunities. **\[2 marks\]**

In [42]:
# Solution
ROE = 0.15
b = 0.6
EPS = 2
r = 0.12

g = ROE * b
print(f"Sustainable Growth Rate: {g:.2%}")

D = EPS*(1-b)
s = D/(r-g)
print(f"Stock Value at the Beginning of 2024: ${s:.2f}")

PVGO = s - EPS/r
print(f"Present Value of Growth Opportunities (PVGO): ${PVGO:.2f}")


Sustainable Growth Rate: 9.00%
Stock Value at the Beginning of 2024: $26.67
Present Value of Growth Opportunities (PVGO): $10.00


## Question 2 \[15 marks\]
One of the businesses owned by Accretion, Nautilus is a machine tool manufacturer and system integrator, specialising in precision hydraulics for large-scale industrial applications.

Nautilus currently has several ground-breaking patents, on the stabilisation of structural columns for large-scale industrial developments in difficult soil conditions. This is planned to revolutionise the industry and be a massive boom for the future of Nautilus. The bond market has not reacted to this reveal of information for a whole year. With a new windfall from a successful year, Nautilus plans to repurchase some bonds it has already issued onto the market, thus you’re tasked to evaluate these issued bonds.

### Exhibit 3: Current Par Yield and Spot Rates
| Maturity | Par Rate (Annual Coupon) | Spot Rate (Annual Coupon) |
| --- | --- | --- |
| 1 year | 2.50% | 2.50% |
| 2 years | 2.99% | 3.00% |
| 3 years | 3.48% | 3.50% |
| 4 years | 3.95% | 4.00% |
| 5 years | 4.37% | ? |

### Exhibit 4: Information for Selected Bonds
| Bond Name | Maturity | Coupon | Type of Bond |
| --- | --- | --- | --- |
| Bond A (Face value $1,000) | 3 years | 6.40% annual | Option-free |
| Bond B | 3 years | 4.30% annual | Callable at par on start of year 1 and tear 2 |
| Bond C | 3 years | 4.30% annual | Putable at par on start of year 1 and year 2 |

### Exhibit 5: Binomial Interest Rate Tree, based on an estimated interest rate volatility of 10%, where ‘u’ represents an up move and ‘d’ represents a down move.
| Year 0 | Year 1 | Year 2 |
| --- | --- | --- |
| 2.2500% (r) | 3.5930% (ru node) | 4.6470% (ruu node) |
|  | 2.9417% (rd node) | 3.8046% (rud node) |
|  |  | 3.1150% (rdd node) |

2 a.) Based on Exhibit 3, find the five-year spot rate. **\[2 marks\]**

In [ ]:
# Solution

2 b.) Assuming the law of one price, use Exhibit 3 to calculate the forward rate of a one-year loan starting in three years. **\[1 mark\]**

In [ ]:
# Solution

2 c.) Given spot rates for one-, two-, and three-year zero bonds, how many forward rates can be calculated? Please list the forward rates that can be calculated and briefly explain your answer. **\[3 marks\]**

In [ ]:
# Solution

2 d.) Find the yield to maturity for Bond A. You can use the IRR function from NumPy Financial. **\[3 marks\]**

In [ ]:
# Solution

2 e.) Based on Exhibit 5, assume an equal probability of interest rate going up and down at each node. Calculate the value of Bond B and Bond C with the binomial tree model. **\[3 marks\]**

In [ ]:
# Solution

2 f.) All else being equal, explain the effect of a fall in interest rates on Bond B and Bond C. **\[2 marks\]**

In [ ]:
# Solution

2 g.) All else being equal, which bond is most likely to increase in value if interest rate volatility is 15% rather than 10%? Briefly explain your answer. (Hint: consider the value of options) **\[1 mark\]**

In [ ]:
# Solution

## Question 3 \[19 marks\]
Uiop, manages the money generated in the conglomerate. As part of your role to be in the know of the many operational parts of Accretion, you’ve been rotated to the strategic options desk within Uiop, and you’ve been asked to refresh your knowledge on derivatives and their pricing, by answering these questions.

Consider a stock that is trading at $100 today. The stock does not generate income/pay dividends. The stock is traded in a well-functioning market with no transaction costs and no restrictions on short sales. Both borrowing and lending can be done in unlimited amounts at the 2% risk-free rate.

3 a.) What is the difference between forward contracts and futures contracts (Answers should be no longer than 200 words) **\[4 marks\]**

In [ ]:
# Solution

3 b.) Consider a futures contract on the stock with a maturity of one year. Suppose that the futures price is currently at $110. Are the futures fairly priced? Describe an arbitrage strategy that would allow you to make a riskless profit. **\[2 marks\]**

In [ ]:
# Solution

3 c.) Same as question b) but suppose that the futures price is currently at $95. Describe your arbitrage strategy. **\[2 marks\]**

In [ ]:
# Solution

Suppose that you hold a long position on a European call option that has an underlying asset price of $57.03, strike price of $55, risk-free rate of 0.22%, 32% volatility, and time-to-expiration of 0.25. The underlying asset does not have any investment yield.

3 d.) Value this call option. **\[1 mark\]**

In [ ]:
# Solution

3 e.) Based on the Black-Scholes-Merton model, describe a portfolio that replicates the call option’s payoff. **\[1 mark\]**

In [ ]:
# Solution

3 f.) Define a function to price the option with the binomial tree method. The function should take the number of steps (n) as one of the inputs. You should NOT use list comprehension in the function. **\[3 marks\]**

In [ ]:
# Solution

3 g.) By setting n = 10, 50 and 100, compare and comment on the results under the two methods. **\[2 marks\]**

In [ ]:
# Solution

When buying two calls with the exercises price of $x_{1}$ and $x_{3}$ and selling two calls with the exercise price of $x_{2}$, where
$$x_{2} = \frac{x_{1}+x_{3}}{2},$$
with the same maturity for the same stock, we call it a butterfly. Consider the following call options for the stock that trades at $57.03:
| Option Name | Strike Price | Call Premium (Price) |
| --- | --- | --- |
| Call Option 1 | 50 | 10 |
| Call Option 2 | 55 | 7 |
| Call Option 3 | 60 | 5 |

3 h.) Create a graphical representation of the butterfly strategy’s payoff. **\[2 marks\]**

In [ ]:
# Solution

3 i.) Why might an investor enter into such a strategy? **\[2 marks\]**

In [ ]:
# Solution

## Question 4 \[13 marks\]
Suppose that you want to build a portfolio with the stocks (with tickers of) AAPL, GOOG, AMZN and META. You plan to retrieve the relevant Yahoo Finance data with the yfinance Python module from the 1st of Jan. 2017 to 1st of Nov. 2023 (inclusive).

4 a.) Calculate the daily returns of these stocks. You should use the adjusted daily closing price. **\[1 mark\]**

In [ ]:
# Solution

4 b) Based on a.), find the covariance matrix of these stocks. **\[1 mark\]**

In [ ]:
# Solution

4 c.) Suppose that the four stocks are equally weighted, find the annualised portfolio expected returns and portfolio variance. **\[2 marks\]**

In [ ]:
# Solution

4 d.) Find the efficient portfolio with the maximum Sharpe ratio. What is the corresponding Sharpe ratio? **\[2 marks\]**

In [ ]:
# Solution

4 e.) Plot the efficient frontier (Without the use of the Plotly library in Python). **\[3 mark\]**

In [ ]:
# Solution

4 f.) Based on the concept of diversification, comment on the current portfolio. How can this portfolio be improved? (Answers should be no longer than 200 words) **\[4 marks\]**

In [ ]:
# Solution

## Question 5 \[14 marks\]
A high-profile client Pam, whom you’ve built a great business relationship with over the years has asked you to quickly double-check some figures for life insurance he has received. You decide to apply the [2015 VBT Unismoke ANB/ALB](http://www.soa.org/files/research/exp-study/2015-vbt-smoker-distinct-alb-anb.xlsx), the Valuation Basic Table (VBT) to using the Age Nearest Birthday (ANB) method, with a valuation interest rate of 5.2% you submit your calculated figures to aid Pam's decision.

Today is 11 November 2023, and the type of life insurance Pam was viewing was permanent life insurance. Pam was born on 25 March 1980, with the health condition standard for her cohort of the same age and gender (Female).

5 a.) Calculate Pam’s annual unconditional survival rates $_{n}p_{x}$ from now to age 54. **\[4 marks\]**

In [ ]:
# Solution

5 b.) Calculate Pam’s life expectancy in years (to one decimal place). **\[3 marks\]**

In [ ]:
# Solution

5 c.) Calculate the probability that Pam dies exactly between 6 and 9 years from now (to five decimal places). **\[3 marks\]**

In [ ]:
# Solution

5 d.) Calculate the minimum annual premium rate (premium as a fraction of death benefit) that your company should charge for Pam's cohort (to five decimal places). **\[4 marks\]**

In [ ]:
# Solution

## Question 6 \[8 marks\]
The conglomerate you are working for, Accretion, decided to digitalize the financial analysis in order to optimize their operations. Since you have expertise in a multitude of financial instruments, you have been tasked to perform the innovation.

Firstly, you are asked to design a tool for computing the zero spot rates using the bootstrap method, and then verify its validity by applying it to Exhibit 6 below.

### Exhibit 6: Spot Rate Table
| Principal | Maturity (Years) | Coupon (per Year) | Price | Coupon Frequency | Zero Rate (Continuous) |
| --- | --- | --- | --- | --- | --- |
| 100 | 0.5 | 0 | 99.8 | 0.5 (SA) | 0.4% |
| 100 | 1 | 4 | 101.2 | 1 (A) | 2.729% |
| 100 | 1.5 | 5 | 102.4 | 0.5 (SA) | 3.353% |

- SA: Semi-annual coupon
- A: Annual coupon

6 a.) Design a function that take in the required parameters for the bootstrap method in form of a Pandas DataFrame and returns the same DataFrame but with a new column called "Zero Rates". Then, check that your function gives the correct spot rates in the table above. **\[4 mark\]**

Here is the formula for finding zero-rates using the bootstrap method as defined in Week 4 exercise class:

$$r(n) = -\frac{1}{n}ln\left(\frac{1}{P+c}\left[B-c\sum^{n-\Delta t}_{t=\Delta t}e^{-r(t)t}\right]\right).$$


I will also test this function on the table from Week 4 Exercise class, to ensure correct behaviour for different input data

| **Principal** | **Maturity (Years)** | **Coupon (per Year)** | **Price** | **Yield** | **Zero Rate (Continuous)** |
| --- | --- | --- | --- | --- | --- |
| 100 | 0.25 | 0 | 99.6 | 1.6064(Q) | 1.603 |
| 100 | 0.5 | 0 | 99.0 | 2.0202(SA) | 2.010 |
| 100 | 1.00 | 0 | 97.8 | 2.2495(A) | 2.225 |
| 100 | 1.50 | 4 | 102.5 | 2.2949(SA) | 2.284 |
| 100 | 2.00 | 5 | 105.0 | 2.4238(SA) | 2.416 |

In [106]:
spot_rate_df = pd.DataFrame([[100, 0.5, 0, 99.8, 0.5], [100, 1, 4, 101.2, 1], [100, 1.5, 5, 102.4, 0.5]],
                            columns=["Principal", "Maturity", "Coupon", "Price", "Coupon Frequency"])

#dataframe for testing (from Week 4 Exercise class)
spot_rate_test_df = pd.DataFrame([
                                    [100, 0.25, 0, 99.6, 0.25],
                                    [100, 0.5, 0, 99.0, 0.5],
                                    [100, 1, 0, 97.8, 1],
                                    [100, 1.5, 4, 102.5, 0.5],
                                    [100, 2, 5, 105.0, 0.5]
                                ],
                                columns=["Principal", "Maturity", "Coupon", "Price", "Coupon Frequency"])

#This works for any coupon frequency (not just annual/semi-annual)
def bootstrap_method(df: pd.DataFrame) -> pd.DataFrame:
    # Solution
    zero_rates = np.zeros(len(df))

    principals = np.array(df["Principal"])
    maturities = np.array(df["Maturity"])
    coupons = np.array(df["Coupon"])
    prices = np.array(df["Price"])
    coupon_frequencies = np.array(df["Coupon Frequency"])

    for i in range(len(df)):
        discount_sum = 0
        
        c_freq = coupon_frequencies[i]
        n_coupons = coupons[i] * c_freq
        n = maturities[i]
        B = prices[i]
        P = principals[i]

        #calculate the sum of discounted coupons
        for m in np.arange(0,maturities[i],c_freq):
            #zero maturity doesn't exit, but to avoid doing two loops with different starting points (0.5 and 1) depending on coupon type, we just skip it
            if(m == 0): 
                continue

            #this uses np.where to find the index of the maturity in the array, and then uses that index to find the corresponding zero rate
            discount_sum += np.exp(-(zero_rates[np.where(maturities == m)[0][0]])*m)
    
        zero_rates[i] = np.round(-np.log((B-n_coupons*(discount_sum))/(P+n_coupons))/n, decimals=5)
        
    df["Zero Rate"] = zero_rates * 100
    
    return df



bootstrap_method(df=spot_rate_df)

,Principal,Maturity,Coupon,Price,Coupon Frequency,Zero Rate
0,100,0.5,0,99.8,0.5,0.400
1,100,1.0,4,101.2,1.0,2.729
2,100,1.5,5,102.4,0.5,3.353


It is often that in real world investors encounter friction costs such as fees, commissions and other types of expenses. Currently, NPV is used to measure ther value of an investment opportunity:

$$NPV = C_{0} + PV.$$

Your task is to design a tool for evaluating net value of a bond, which takes into consideration accrued interest, commission fees on entrance and exit (assume there is commission for settling the return of the principal), and administrative fees, which are monthly fees associated with the exchange that keeps track of asset ownership.

6 b.) Define a python function for calculating the net value of a bond subject to accrued interest, entry and exit fees and administrative fees (assume the administrative fees are paid out at the same frequency as the coupons). The accrued interest MUST be computed from the coupon value. **\[3 marks\]**

In [112]:
def improved_npv(principal: float, coupon: float, price: float, interest_rate: float, n_periods:float, administrative_fee: float, entry_fee: float,
                 exit_fee: float, days_from_last_coupon: int, coupon_frequency_per_year: float) -> float:
    # Solution
    init_invesment = price + entry_fee
    
    PV_coupons = 0
    N = n_periods / coupon_frequency_per_year

    for t in range(1, int(N+1)):
        PV_coupons += (coupon - administrative_fee) / ((1 + interest_rate * coupon_frequency_per_year) ** (t))

    PV_principal = principal / ((1 + interest_rate * coupon_frequency_per_year) ** (N))
    discounted_exit_fee = exit_fee / ((1 + interest_rate * coupon_frequency_per_year) ** (N))

    accrued_interest = coupon * (days_from_last_coupon / (365 * coupon_frequency_per_year))
    
    PV = PV_coupons + PV_principal - accrued_interest - discounted_exit_fee
    
    NPV = -init_invesment + PV

    return  NPV

6 c.) Using the values from Exhibit 7 compute the net value of the investment. Is this a good investment considering all the expenses? **\[1 mark\]**

### Exhibit 7: Information for an Investment
| Parameter | Value |
| --- | --- |
| Principal | $100 |
| Coupon | $4 (Yearly) |
| Price | $101.5 |
| Interest Rate | 2% |
| Maturity | 3 years |
| Administrative Fee | $1 (Yearly) |
| Entry Fee | $5 |
| Exit Fee | $5 |
| Days from Last Coupon | 200 |

In [113]:
# Solution
NPV = improved_npv(principal=100, coupon=4, price=101.5, interest_rate=0.02, n_periods=3, administrative_fee=1, entry_fee=5, exit_fee=5, days_from_last_coupon=200, coupon_frequency_per_year=1.0)
print("NPV of the bond:", round(NPV, 2))

NPV of the bond: -10.52


It's not a really good investment due to the negative NPV value